#                              Biomedical Image Processing
##                 Dash Web Application   

### Aim: To build a web application using dash which can be used as an user inteface for the brain tumor classification model. 

- Step1: Importing required packages for the model and dash application building

In [1]:
# Requred Packages for Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
import warnings
warnings.simplefilter("ignore")
import os
from os import listdir
import glob
import cv2
import imutils
import keras
from skimage import io
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization, Activation,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from PIL import Image, ImageOps
import scipy.misc
from skimage.transform import rotate, AffineTransform, warp
from keras.models import Sequential

# Required packages for Dash App
from sklearn.preprocessing import OneHotEncoder
import time
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import re
import pybase64
import base64
import io
import resnet



In C:\Users\anish\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\anish\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\anish\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\anish\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\anish\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

- Step2: Dash App Creation:
    - 1: Application Building: Front end development of the web application
        - Page1: Aim of the project
        - Page2: Classification model
        - Page3: About the team and mentor

    - 2: Model Incorporation:
        - Image processing with the predefined preprocessing steps and model (hdf5 file)
        - Callback tagging for the prediction output

    - 3: Deployment:
        - Production environment 
        - Dash server with port number - 8050
        
###### Please run the below codes and access http://127.0.0.1:8050/ for opening the dash web application
     Note: The python file kernel has to be running while working on the web application. 


In [2]:
 def names(number):
    if(number == 0):
        return 'A tumor has been detected which may be a Glioma tumor '
    elif(number == 1):
        return 'A tumor has been detected which may be a Menengioma tumor '
    elif(number == 2):
        return 'No tumor has been detected'
    elif(number == 3):
        return 'A tumor has been detected which may be a Pitiutary tumor'
          
        
#--------- Application Building -----------#

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title= "AI Bioanalytical Tool"


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "font-family": "Algerian",
    "font-size": 20,
    "text-align": "center",
    "position": "absolute",
    "top": 0,
    "left":10,
    "bottom": 0,
    "width": "10rem",
    "margin-left": "1rem",
    "padding": "1rem 1rem",
    'background-image': 'linear-gradient(DarkSlateGrey, DarkSlateBlue)',
    'color' :  'Red !important'
   
}


# the styles for the main content position it to the right of the sidebar and

CONTENT_STYLE = {
    "margin-left": "12.6rem",
    "margin-right": "5rem",
    
}

sidebar = html.Div(
    [html.Div(style={'position':'absolute', 'left':'-10px', 'top':'0','width': '3%','height': '100%',
                    'lineHeight': '100%',
                    'background-image': 'linear-gradient(black,maroon)'
            }),
        html.Div(style={'position':'absolute', 'right':'-10px', 'top':'0','width': '3%','height': '100%',
                    'lineHeight': '100%','borderWidth': '1px','borderStyle': 'black', 
                    'borderRadius': '5px',
                    'background-image': 'linear-gradient(black,maroon)'
            }),
     html.Div("Team:AI Triad", style= {'position': 'absolute', 'color': 'white', 'left': '-1',
                                       'top': '545px','textAlign': 'center','font-family': 'Algerian',
                                       'font-size':'medium'
                                       }),
     
     html.Img(src='https://catking.in/wp-content/uploads/2018/04/1200px-IIM_Lucknow_Logo.svg_.png', 
              style={'height':'120px', 'width':'120px', 'margin-left': '0.4rem'}),
     

        html.Hr(),

        dbc.Nav(
            [
                dbc.NavLink("Home", href="/page-1", id="page-1-link"),
                dbc.NavLink("Model", href="/page-2", id="page-2-link"),
                dbc.NavLink("About Us", href="/page-3", id="page-3-link")
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

            

content = html.Div(id="page-content", style=CONTENT_STYLE)
app.config['suppress_callback_exceptions'] = True
app.config.suppress_callback_exceptions = True
app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


@app.callback(
    [Output(f"page-{i}-link", "active") for i in range(1, 4)],
    [Input("url", "pathname")],
)

def toggle_active_links(pathname):
    if pathname == "/":
        # Treat page 1 as the homepage / index
        return True, False, False
    return [pathname == f"/page-{i}" for i in range(1, 4)]


@app.callback(dash.dependencies.Output("page-content", "children"), 
              [dash.dependencies.Input("url", "pathname")])




def render_page_content(pathname):
    if pathname in ["/", "/page-1"]:
        return [html.Div('Bio-medical Image Processing: Brain Tumor Classification', 
            style={'position':'absolute','textAlign': 'center','font-size': 'xx-large', "color":"white",
                   'background-image': 'linear-gradient(DarkSlateGrey, DarkSlateBlue)','width':'84%'
                  }),
                html.Img(src='https://learningbreakthrough.com/wp-content/uploads/2007/06/Fotolia_63666932_Subscription_Monthly_M.jpg', 
              style={"width": "500px", 'position': 'absolute','top':'65px','height': '500px', "left": '700px'}),
                
                html.Div("Brain tumor detection and classification is a very complex task which requires high accuracy and precision.\
                Even a minute error by the doctors while diagnosing the tumor might cost a life. This bio-analytical tool is an easy\
                to use interface for the analysis of magnetic resonance brain images designed for the doctors\clinician to aid them\
                for classifying accurately the 3 major types of brain tumors which are; Glioma, Menengioma and Pituatary.",
                         style= {'position': 'absolute','width': "510px",'left':'300px', 'top':'135px',"font-family": "Verdana" }),
                html.Div("Deep Learning has paved a way to automate the entire process of this bio-analytical tool by artificial\
                intelligence which is a big step forward in personalizing treatments and ensuring better patient outcomes.",
                         style= {'position': 'absolute','width': "500px",'left':'300px', 'top':'330px',"font-family": "Verdana" }),
                html.Div("Doctors\clinician can upload a brain MRI image (tumor/non-tumor) and get a report on the detection and\
                classification of the image with a high accuracy rate as well as the probability distribution which can guide them to\
                improve their judgement.   ",
                         style= {'position': 'absolute','width': "515px",'left':'300px', 'top':'430px',"font-family": "Verdana" })
               ] 
            
    
    elif pathname == "/page-2":
        return [html.Div('Bio-medical Image Processing: Brain Tumor Classification', 
            style={'position':'absolute','textAlign': 'center','font-size': 'xx-large', "color":"white",
                   'background-image': 'linear-gradient(DarkSlateGrey, DarkSlateBlue)','width':'84%'
                  }),
                html.Div([dcc.Upload(
                    id='upload-image',
                    children=html.Div([html.A('Upload Files here')]
        ),
        style={'position':'absolute','textAlign': 'center', 'top':'60px','border-style': 'groove',
               'border-width': '10px','background-color': 'lightgrey','font-weight': 'bold'
              
              }
                    , multiple=True),
                html.Div(id='output-image-upload', style={'position':'absolute','textAlign': 'left',
                                                      'top':'110px', 'border-style': 'solid',
                                                        'border-width': '2px','border-color': 'black' })]),
                html.Div(id='prediction', style={'position':'absolute','left':'580px', 'top':'160px', 
                                                 'width': '670px','font-size':'x-large',
                                                 }),
                html.Div(id='prediction2', style={'position':'absolute', 'left':'370px', 'top':'430px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                html.Div(id='prediction3', style={'position':'absolute', 'left':'540px', 'top':'430px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                html.Div(id='prediction4', style={'position':'absolute', 'left':'710px', 'top':'430px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                html.Div(id='facts', style={'position':'absolute', 
                                               'left':'580px', 'top':'230px', 
                                               'font-size': 'large','height': '200px',
                                               'width': '600px'}),
                html.Div("Upload a brain MRI Image which has a good resolution and wait for the results!",
                        style= {'position':'absolute', 'left': '353px','top':'62px', 'border': 'solid ',
                                 'background-color': 'black','border-width': '5px','border-color': 'black',
                                'color':'white', 'font-family': 'Verdana'
                                }),
                html.Div([html.Div(" ", style = {'position':'absolute', 'left':'200px', 'top':'400px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                          html.Div(" ", style = {'position':'absolute', 'left':'370px', 'top':'400px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                          html.Div(" ", style = {'position':'absolute', 'left':'540px', 'top':'400px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                          html.Div(" ", style = {'position':'absolute', 'left':'710px', 'top':'400px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                    
                          html.Div(" ", style = {'position':'absolute', 'left':'200px', 'top':'430px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                          html.Div(" ", style = {'position':'absolute', 'left':'370px', 'top':'430px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                          html.Div(" ", style = {'position':'absolute', 'left':'540px', 'top':'430px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                          html.Div(" ", style = {'position':'absolute', 'left':'710px', 'top':'430px',
                                                   'height': '5%','lineHeight': '7%','border-style': 'solid',
                                                   'border-color': 'black',
                                                    'textAlign': 'center', 'font-size': '44',
                                                     'line-height': '0.8',
                                                     'font-family':'Arial Black','width': "170px"}),
                      ]),
                html.Div([html.Div("Tumor Classes ", style = {'position':'absolute', 'left':'200px', 'top':'400px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                          html.Div("Glioma", style = {'position':'absolute', 'left':'370px', 'top':'400px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                          html.Div("Menengioma", style = {'position':'absolute', 'left':'540px', 'top':'400px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                          html.Div("Pitiuitary", style = {'position':'absolute', 'left':'710px', 'top':'400px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"}),
                          html.Div("Probability in %", style = {'position':'absolute', 'left':'200px', 'top':'430px',
                                                'textAlign': 'center', 'font-size': '20',
                                                 'font-family':'Arial Black','width': "170px"})]),
                 html.Div([html.Div("Probability distribution: Probability distribution is the weightage for each class generated for the given\
                 image. With the combination of the model accuracy and probability distribution, doctor\clinician can make a\
                 more accurate diagnosis."),
                           html.Div("Disclaimer: The user should upload .jpg image which has a high resolution and has not been\
                           edited in any form."),
                          html.Div("This is a pilot project and is not suggested to use it for diagnosis whithout prior approval!")]
                           , style={'position':'absolute', 'left':'200px', 'top':'475px','textAlign': 'left', 
                                    'font-size': '20','font-family':'Verdana','width': "1000px",'border': 'solid ',
                                    'background-color': 'black','border-width': '5px','border-color': 'black',
                                    'color':'white'})
                          
                          
                         
               ]
    
    elif pathname == "/page-3":
        return [html.Div('Bio-medical Image Processing: Brain Tumor Classification', 
            style={'position':'absolute','textAlign': 'center','font-size': 'xx-large', "color":"white",
                   'background-image': 'linear-gradient(DarkSlateGrey, DarkSlateBlue)','width':'84%'
                  }),
                html.Div("About Us: Team AI Triad",style= {'position':'absolute', 'left':'200px', 'top':'55px',
                                                           'textAlign': 'center', 'font-size': '44',
                                                           'text-decoration': 'underline', 'line-height': '0.8',
                                                            'font-family':'Arial Black','width': "84%"} ),
                html.Div("We are working professionals who are a part of the Management Development Program;\
                “Executive Program In Data Science-2020” conducted by IIM, Lucknow and Eruditus Executive Education (as a\
                technical partner).", 
                         style = {'position':'absolute', 'left':'210px','top':'70px', 'font-family':'Verdana', 'width': '80%',
                                 'font-size': '16'}),
                 dbc.CardImg(src="https://i.ibb.co/mqhgFQ9/DS.jpg", 
                    style={"width": "120px","height": "120px", 'position': 'absolute','top':'180px', 'left':'310px',
                          'border-radius': '50%','textAlign': 'center'}),
                
                html.Div("Deepak Sharma, a guest faculty of Data Science at his alma mater IIM Lucknow, is an expert\
                in Artificial Intelligence using Machine Learning, Deep Learning and NLP. He is an MBA Graduate with\
                multi-domain industrial experience in Healthcare, Supply Chain, Telecom, Work Force Management of\
                total 17 years out of with 10 years of Analytical experience explicitly.He has worked as Data\
                Scientist in Teradata, Beckon Dickinson,  CSC (DXC) and Cap Gemini. Currently, he is running a startup\
                RecogX.AI which is an Artificial Intelligence related venture using Computer Vision.", 
                        style= {'position':'absolute', 'left':'220px','top':'325px', 'font-family':'Verdana', 'font-size': 'small',
                                 'width': "350px"}),
               html.Div([html.Div(" ")], style = {'position':'absolute', 'left':'207px', 'top':'130px',
                                                                  'height': '76%','lineHeight': '77%','lineWidth': '362px','border-style': 'double',
                                                                   'border-radius': '40px','border-color': 'DarkSlateBlue',
                                                                   'textAlign': 'center', 'font-size': '44',
                                                                 'text-decoration': 'underline', 'line-height': '0.8',
                                                                  'font-family':'Arial Black','width': "360px"}),
                html.Div ("Project Mentor:Deepak Sharma", style = {'position': 'absolute', 'left':'207px', 'top':'140px',
                                                   'textAlign': 'center', 'font-size': '44','text-decoration': 'underline', 
                                                    'line-height': '0.8','font-family':'Arial Black','width': "360px"}),
                html.Div([html.Div(" ")], style = {'position':'absolute', 'left':'576px', 'top':'130px',
                                                                  'height': '76%','lineHeight': '77%','border-style': 'double',
                                                                   'border-radius': '40px','border-color': 'tomato',
                                                                   'textAlign': 'center', 'font-size': '44',
                                                                 'text-decoration': 'underline', 'line-height': '0.8',
                                                                  'font-family':'Arial Black','width': "690px"}),
                dbc.CardImg(src="https://i.ibb.co/1fG9ZMh/Anitha.jpg", 
                    style={"width": "120px", 'position': 'absolute','top':'135px', "left": '610px',
                          'border-radius': '50%'}),
                html.Div ("Anitha Madhusudhan", style = {'position': 'absolute', 'top': '250px', 'left': '580px',
                                                        'font-family':'Arial Black','font-size': '44'}),
                dbc.CardImg(src="https://i.ibb.co/j528THW/Ashitha.jpg", 
                    style={"width": "120px", 'height':'120px','position': 'absolute','top':'280px', "left": '610px',
                          'border-radius': '50%'}),
                html.Div ("Ashitha Menon", style = {'position': 'absolute', 'top': '400px', 'left': '605px',
                                                    'font-family':'Arial Black','font-size': '44'}),
                dbc.CardImg(src="https://i.ibb.co/3NyXdmw/kavya.jpg", 
                    style={"width": "120px", 'height':'120px','position': 'absolute','top':'430px', "left": '610px',
                          'border-radius': '50%'}),
                html.Div ("Kavya Brungi", style = {'position': 'absolute', 'top': '545px', 'left': '608px',
                                                    'font-family':'Arial Black','font-size': '44'}),
                html.Div("Anitha is a biotechnologist, who is an MSc Graduate and worked as Senior Research Associate\
                in a drug discovery organization, BBRC. She has multi-domain knowledge which includes compound Management,\
                Invitro development and analysis and Technology Platform. She is an active participant of the EPDS program\
                from IIM, Lucknow. Her main focus is to understand the statistical concepts, cutting-edge tools and\
                techniques which leads to Data Driven Decisions.", 
                        style= {'position':'absolute', 'left':'770px', 'top':'140px', 'font-family':'Verdana', 
                                'font-size': 'small','width': "485px",'border-bottom':'dotted','border-color': 'tomato'}),
                html.Div("Ashitha comes from a background in Academics which she pursued post her M.Phil in Commerce.\
                During her research, it became evident to her that data science would play a key role in cultivating \
                         an analytical mind-set to develop data-driven solutions to difficult business challenges.\
                         She is currently pursuing her Executive program in Data Science at the prestigious IIM\
                         Lucknow where she is learning various concepts and computer languages to draw insights\
                         from large datasets", 
                        style= {'position':'absolute', 'left':'770px', 'top':'280px', 'font-family':'Verdana', 
                                'font-size': 'small','width': "485px",'border-bottom':'dotted','border-color': 'tomato'}),
                html.Div("Kavya Brungi working as assistant manager in the Advanced Analytics team of Tata Motors\
                Finance after completing her PGDM. At Tata Motors Finance she has a part in building various\
                projects related to financing the customers and predicting the payment patterns of the customers", 
                        style= {'position':'absolute', 'left':'770px', 'top':'430px', 'font-family':'Verdana', 
                                'font-size': 'small','width': "485px"})
                
               ]
                
               
               
               
               
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

def parse_contents(contents):
    return html.Img(src=contents, style={'height':'250px', 'width':'250px'})

@app.callback([dash.dependencies.Output('output-image-upload', 'children'),
               dash.dependencies.Output('prediction', 'children'), 
               dash.dependencies.Output('prediction2', 'children'),
               dash.dependencies.Output('prediction3', 'children'),
               dash.dependencies.Output('prediction4', 'children'),
               dash.dependencies.Output('facts', 'children')],             
              [dash.dependencies.Input('upload-image', 'contents')])

#--------- Model Incorporation and Deployment-----------#
            
def update_output(list_of_contents):        
    
    #Only predicts for the first image inserted (list_of_contents[0])
    if list_of_contents is not None:
        children = parse_contents(list_of_contents[0])
        
        #Transforming the image from base64 
        img_data = list_of_contents[0]
        img_data = re.sub('data:image/jpeg;base64,', '', img_data)
        img_data = base64.b64decode(img_data)  
        
        stream = io.BytesIO(img_data)
        img_pil = Image.open(stream)

        
        
        #Load model, change image to array and predict
          
        dim = (150, 150)
          
        img = np.array(img_pil.resize(dim))
        img=cv2.cvtColor((img), cv2.COLOR_BGR2GRAY ) 
        img= 255-img
        x = img.reshape(-1,150,150,1)
        
        classifier= load_model('AKA.h5') # saved model
        answ = classifier.predict(x) # predicted array
        classification =np.argmax(answ,axis=1)[0] # argmax conversion
        Conf_level =pd.DataFrame(answ) # conversion to dataframe just for indexing purpose
        
        # Prediction 1: using the function named "names" defined earlier
        pred= names(classification) 
    
        
        #probability distribution and facts for the respective classes. 
        if classification==0:
            facts = 'Glioma is a type of tumor that occurs in the brain and spinal cord.\
                    A glioma can affect your brain function and be life-threatening depending on\
                    its location and rate of growth. Gliomas are one of the most common types of primary brain tumors.'
            pred2= str(round((Conf_level[0][0]*100) ,2)) + "%"
            pred3= str(round((Conf_level[1][0]*100) ,2)) + "%"
            pred4= str(round((Conf_level[3][0]*100) ,2)) + "%"
        elif classification==1:
            facts = 'A meningioma is a tumor that arises from the meninges, the membranes that surround your brain.\
                    Most meningiomas grow very slowly, often over many years without causing symptoms.\
                    They occur more commonly in women.'
            pred2= str(round((Conf_level[0][0]*100) ,2)) + "%"
            pred3= str(round((Conf_level[1][0]*100) ,2)) + "%"
            pred4= str(round((Conf_level[3][0]*100) ,2)) + "%"
            
        elif classification==3:
            facts = 'Pituitary tumors are abnormal growths that develop in your pituitary gland.\
                    Most pituitary tumors are noncancerous (benign) growths that remain in your pituitary\
                    gland or surrounding tissues.'
            
            pred2= str(round((Conf_level[0][0]*100) ,2)) + "%"
            pred3= str(round((Conf_level[1][0]*100) ,2)) + "%"
            pred4= str(round((Conf_level[3][0]*100) ,2)) + "%"
        
        else:
            facts=None
            pred2 = None
            pred3 = None
            pred4 = None
      
        return children, pred, pred2,pred3,pred4, facts    
    else:
        return ("no_update"," "," "," "," "," ")       
  
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
